<a href="https://colab.research.google.com/github/SBGnanapriya/rotary_screw/blob/main/compressor_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

import joblib

In [4]:
df = pd.read_csv("compressor_data.csv")
df.head()

,Motor_Current_A,Oil_Temperature_C,Line_Pressure_bar,Filter_DeltaP_bar,Vibration_RMS_mm_s,Running_Hours,Status,Label,Reason
0,38.90,89.29,3.08,0.225,2.42,2156.0,Warning,1,Service interval warning (2000-3000 hrs)
1,59.33,94.59,2.90,0.237,4.81,1639.5,Warning,1,Multiple parameters in warning range
2,34.68,104.85,3.08,0.532,4.25,511.6,Critical,2,Filter choking (High temperature + High ΔP)
3,26.02,99.59,2.76,0.572,5.26,4040.1,Critical,2,Service critical (>3000 hrs) - Service required
4,27.87,66.94,2.82,0.047,1.59,542.7,Normal,0,NaN
